In [21]:
# https://wearebraindead.com/collections/pant?sort_by=
import requests
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time
SCROLL_PAUSE_TIME = 1.2

product_url = []
image_url = []

URL = 'https://wearebraindead.com/collections/all?sort_by='
driver = webdriver.Chrome('/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
driver.get(URL)

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(driver.page_source,'html.parser')

for i in soup.find_all('div',{'class':'item'}):
    product_url.append('https://wearebraindead.com/' + i.a.get('href'))
    image_url.append('https:' + i.find('div',{'class':'product-img'}).img.get('src'))

In [27]:
import pandas as pd

first_pass = pd.DataFrame(columns = ['product_url', 'image_url'])
first_pass.product_url = product_url
first_pass.image_url = image_url
#first_pass.to_csv('brain-dead-first-pass.csv', encoding='utf-8-sig')

# Second Pass: Price, Product Name, Description...

In [28]:
from bs4 import BeautifulSoup
import urllib.request
from datetime import date
import pandas as pd
date_today = date.today()

In [96]:
name = []
sales_price = []
price = []
brand = []
product_type = []
timestamp  = []

color = []
product_description = []
material = []

for link in product_url:
    url = link
    ourUrl = urllib.request.urlopen(url)
    soup = BeautifulSoup(ourUrl,'html.parser')
    sales_price.append(str(soup.find('dd',{'class':'product-price'}).text).strip())
    price.append(soup.find('p',{'class':'price'}).text)
    name.append(soup.find('h1',{'class':'product-title'}).text)
    brand.append('Brain Dead')
    product_type.append('Clothing')
    timestamp.append(date_today)
    
    index_des = str(soup.find('div',{'class':'description rte'}).p.text).find('PLEASE NOTE: ALL SALES ARE FINAL.')
    product_description.append(str(soup.find('div',{'class':'description rte'}).p.text).strip())
    
    if str(soup.find('h1',{'class':'product-title'}).text).find(" - ")>0:
        index_color = (str(soup.find('h1',{'class':'product-title'}).text).find(" - "))
        color.append(str(soup.find('h1',{'class':'product-title'}).text)[index_color+3:])
    else:
        color.append(None)
    #index_dimen = str(soup.find('div',{'class':'description rte'}).p.text).find("Dimensions")
    material.append(None)

In [97]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp"]
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = image_url
second_pass.timestamp = timestamp

In [98]:
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp
0,Brain Dead,Clothing,Modernica x Brain Dead Headcase Study Table - ...,Modernica custom Brain Dead Logo head Table in...,,"$ 1,200.00 USD",Maple,None,https://wearebraindead.com//collections/all/pr...,https://cdn.shopify.com/s/files/1/0642/8893/pr...,2020-12-03
1,Brain Dead,Clothing,Brain Research Tea Towel 3 Pack - Blue/Yellow/...,Brain Research Tea Towels. Three cotton tea to...,,$ 25.00 USD,Blue/Yellow/Green,None,https://wearebraindead.com//collections/all/pr...,https://cdn.shopify.com/s/files/1/0642/8893/pr...,2020-12-03
2,Brain Dead,Clothing,Brain Dead Mo-Tools Wood Inlay Hammer Garden T...,"Hand crated red oak wooden handle, polished st...",,$ 90.00 USD,Red Oak/Silver,None,https://wearebraindead.com//collections/all/pr...,https://cdn.shopify.com/s/files/1/0642/8893/pr...,2020-12-03
3,Brain Dead,Clothing,Brain Dead x Cowply Life Celebration Complete ...,Brain Dead x Cowply. Cowply is a skater owned ...,,$ 120.00 USD,Multi,None,https://wearebraindead.com//collections/all/pr...,https://cdn.shopify.com/s/files/1/0642/8893/pr...,2020-12-03
4,Brain Dead,Clothing,Sees All Puzzle Assisted Living 1000 Pc Jigsaw...,Sees All 1000 pc jigsaw puzzle. Brain Dead art...,,$ 45.00 USD,Red/Multi,None,https://wearebraindead.com//collections/all/pr...,https://cdn.shopify.com/s/files/1/0642/8893/pr...,2020-12-03


In [99]:
second_pass.to_csv('brain-dead.csv', index=False, encoding = 'utf-8-sig')

In [100]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/brain-dead.csv', encoding='utf-8-sig')

os.mkdir('brain-dead')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('brain-dead/'+ "brain-dead_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1